In [1]:
%pip install -q kor markdownify requests pydantic pydantic[email] openai colorama bs4 rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.2 MB/s eta 0:00:00


## After Validation

Custom validators serve as transformation functions that, during deserialization, receive input and return the corresponding deserialized value. Beyond validation checks, these validators also play a role in transforming data. It's essential to recognize that they can be utilized for validation only, deserialization only, or both. In the upcoming content, we'll explore the process of defining custom validators using a decorator.

In [2]:
from pydantic import BaseModel, Field, field_validator, ValidationError
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        return value  # custom validators must return a value

try:
    Model(number=12)
except ValidationError as ex:
    print(ex)

1 validation error for Model
number
  Input should be less than 10 [type=less_than, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/less_than


To indicate a validation error in our custom validator, we just raise a ValueError with an appropriate message.

In [3]:
# from pydantic import BaseModel, ConfigDict, Field, field_validator, ValidationError
# from datetime import datetime, timezone
# from typing import List,Dict,Set,Tuple, Union, Annotated, get_args, Any, TypeVar

class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        if value % 2 == 0:
            # number is even, so return it
            return value
        raise ValueError("value must be even")

try:
    Model(number=3)
except Exception as ex:
    print(f"{type(ex)=}, {ex}")

Running custom validator
value=3, type(value)=<class 'int'>
type(ex)=<class 'pydantic_core._pydantic_core.ValidationError'>, 1 validation error for Model
number
  Value error, value must be even [type=value_error, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error


Now, let's make a model that ensures datetimes are always stored as UTC aware datetimes in our model.

In [4]:
from datetime import datetime
import pytz

class Model(BaseModel):
    dt: datetime

    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt

Model(dt="2020-01-01T03:00:00")


Model(dt=datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>))

As we saw in the lecture video, the order of execution of after validators is the top to bottom definition order of the functions in the class.

In [5]:
class Model(BaseModel):
    number: int

    @field_validator("number")
    @classmethod
    def add_1(cls, value: int):
        print(f"running add_1({value}) -> {value + 1}")
        return value + 1

    @field_validator("number")
    @classmethod
    def add_2(cls, value: int):
        print(f"running add_2({value}) -> {value + 2}")
        return value + 2

    @field_validator("number")
    @classmethod
    def add_3(cls, value: int):
        print(f"running add_3({value}) -> {value + 3}")
        return value + 3

In [6]:
Model(number=1)

running add_1(1) -> 2
running add_2(2) -> 4
running add_3(4) -> 7


Model(number=7)

In [7]:
class Model(BaseModel):
    unit_cost: float
    unit_price: float

    @field_validator("*")
    @classmethod
    def round_2(cls, value: float) -> float:
        return round(value, 2)

Model(unit_cost=2.12345, unit_price=5.9876)

Model(unit_cost=2.12, unit_price=5.99)

## Before Validation

Pre-validators, also known as "before validators," allow us to intercept data before Pydantic performs its own validation. Typically, validators run after Pydantic's initial deserialization, ensuring the received value is of the correct type and meets specified model validations. However, in certain cases, employing before validators becomes essential for customizing the deserialization process before Pydantic executes its validation. For instance, this is particularly useful when dealing with a model containing a datetime field, where Pydantic's string-to-datetime coercion requires a specific ISO format.

In [8]:
class Model(BaseModel):
    dt: datetime

Model(dt="2020-01-01T12:00:00")

Model(dt=datetime.datetime(2020, 1, 1, 12, 0))

Let's first write some Python code that will be able to parse these various date strings into proper datetime objects.

We'll use the python-dateutil 3rd party library to do this.

You'll need to install in in your virtual environment, and docs for that library are located here

In particular, we'll use that library's parser.

In [9]:
from dateutil.parser import parse

parse("2020/1/1 3pm")

datetime.datetime(2020, 1, 1, 15, 0)

In [10]:
try:
    parse(datetime(2020, 1, 1, 15, 0, 0))
except TypeError as ex:
    print(ex)

Parser must be a string or character stream, not datetime


Technically we don't have to return the final (model) type from our validator, since the Pydantic validators will run after, but we could if we wanted to. In this case, we're going to attempt to parse the value if it is a string, otherwise we'll just forward the value, whatever type it is, and let Pydantic handle non-string input values.

In [11]:
from typing import Any


class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, value: Any):
        if isinstance(value, str):
            print("parsing string")
            try:
                return parse(value)
            except Exception as ex:
                raise ValueError(str(ex))
        print("pass through...")
        return value

Model(dt="2020/1/1 3pm")

parsing string


Model(dt=datetime.datetime(2020, 1, 1, 15, 0))

## Combining Before and After Validators
We saw how to define before validators, and after validators.

We can define as many before and after validators as we want, and we can also mix both before and after validators.

The before validators will be executed first (bottom to top order), then the Pydantic validator, then the after validators (top to bottom).

Using both before and after validators can be quite handy.

In [12]:
from datetime import datetime
from typing import Any

import pytz
from dateutil.parser import parse
from pydantic import BaseModel, field_validator, ValidationError


class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, value: Any):
        if isinstance(value, str):
            try:
                return parse(value)
            except Exception as ex:
                raise ValueError(str(ex))
        return value

Model(dt=100_000)

Model(dt=datetime.datetime(1970, 1, 2, 3, 46, 40, tzinfo=TzInfo(UTC)))

So now, we are going to combine both of those validators:

In [13]:
class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, value: Any):
        if isinstance(value, str):
            try:
                return parse(value)
            except Exception as ex:
                raise ValueError(str(ex))
        return value

    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt

Model(dt=100_000)

Model(dt=datetime.datetime(1970, 1, 2, 3, 46, 40, tzinfo=<UTC>))

## EXTRA EXAMPLES:


In [14]:
from datetime import date
from pydantic import model_validator
class ScheduledCourse(BaseModel):
    department: str
    course_number: int
    start_date: date
    end_date: date

    @model_validator(mode="before")
    @classmethod
    def validate_dates(cls, data: dict):
        if data["start_date"] > data["end_date"]:
            raise ValueError("Start date must be before end date for the course")
        return data



In [15]:
try:
    course = ScheduledCourse(
        start_date="2024-12-22",
        end_date="2024-11-10"
        )
    print(course)
except ValidationError as e:
    print(e)

1 validation error for ScheduledCourse
  Value error, Start date must be before end date for the course [type=value_error, input_value={'start_date': '2024-12-2...end_date': '2024-11-10'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error


## Custom Validators using Annotations

In [17]:
from pydantic import BaseModel, Field, ValidationError

class Model(BaseModel):
    number: int = Field(gt=0, lt=5)

Or we could do it using an annotated type:

In [16]:
from typing import Annotated

BoundedInt = Annotated[int, Field(gt=0, lt=5)]

class Model(BaseModel):
    number: BoundedInt

We can do something similar with validators.

First, we define our validation function. Because we are essentially defining this function outside of a class, it is a regular function, not a class method (so we don't need that cls argument at all).

Let's do our datetime example, starting with a before validator:

In [19]:
from datetime import datetime
from typing import Any
from pydantic import BeforeValidator, AfterValidator

from dateutil.parser import parse

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

DateTime = Annotated[datetime, BeforeValidator(parse_datetime)]

class Model(BaseModel):
    dt: DateTime

Model(dt="2020/1/1 3pm")


Model(dt=datetime.datetime(2020, 1, 1, 15, 0))

In [20]:
import pytz

def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [21]:
class Model(BaseModel):
    dt: DateTimeUTC

Model(dt="2020/1/1 3pm")

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

Let's look at another example of using annotations for validators.

Suppose we want to define a field that is a list, of some type, that only contains unique elements.

We'll want to make it reusable, so we'll implement this using annotations.

First, we'll start with an annotated type for just integers, then we'll use the same technique I showed you earlier with TypeVar to extend this to arbitrary types.

In [22]:
def are_elements_unique(values: list[Any]) -> list[Any]:
    unique_elements = []
    for value in values:
        if value in unique_elements:
            raise ValueError("elements must be unique")
        unique_elements.append(value)
    return values

UniqueIntegerList = Annotated[list[int], AfterValidator(are_elements_unique)]

class Model(BaseModel):
    numbers: UniqueIntegerList = []

try:
    Model(numbers=[1, 1, 2, 3])
except ValidationError as ex:
    print(ex)

1 validation error for Model
numbers
  Value error, elements must be unique [type=value_error, input_value=[1, 1, 2, 3], input_type=list]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error


## Dependent Field Validations

In [23]:
from pydantic import BaseModel, field_validator, ValidationError, ValidationInfo

class Model(BaseModel):
    field_1: int
    field_2: list[int]
    field_3: str
    field_4: list[str]

    @field_validator("field_3")
    @classmethod
    def validator(cls, value: str, validated_values: ValidationInfo):
        print(f"{value=}")
        print(f"{validated_values=}")
        return value

Model(field_1=100, field_2=[1, 2, 3], field_3="python", field_4=["a", "b"])

value='python'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 100, 'field_2': [1, 2, 3]}, field_name='field_3')


Model(field_1=100, field_2=[1, 2, 3], field_3='python', field_4=['a', 'b'])

Let's see a typical application of this.

Suppose we have a model with a start and end datetime - our validation needs to ensure that the end date is not earlier than the start date, so let's implement that.

We'll bring back the validators we had when dealing with datetimes:

In [24]:
from datetime import datetime
from typing import Annotated, Any

import pytz
from dateutil.parser import parse
from pydantic import AfterValidator, BeforeValidator

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value


def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [25]:
class Model(BaseModel):
    start_dt: DateTimeUTC
    end_dt: DateTimeUTC

    @field_validator("end_dt")
    @classmethod
    def validate_end_after_start_dt(cls, value: datetime, values: ValidationInfo):
        data = values.data
        if "start_dt" in data:
            if value <= data["start_dt"]:
                raise ValueError("end_dt must come after start_dt")
        # if start_dt failed validation, there's not much we can check here.
        #    So just return value as-is
        return value

In [26]:
Model(start_dt="2020/1/1", end_dt="2020/12/31")

Model(start_dt=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>), end_dt=datetime.datetime(2020, 12, 31, 0, 0, tzinfo=<UTC>))

## Project Specs
This is where we left off in the previous section project:

In [27]:
from datetime import date
from enum import Enum
from typing import Annotated, TypeVar
from uuid import uuid4
from pydantic import BaseModel, ConfigDict, Field, field_serializer
from pydantic.alias_generators import to_camel
from pydantic import UUID4


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


T = TypeVar('T')
BoundedString = Annotated[str, Field(min_length=2, max_length=50)]
BoundedList = Annotated[list[T], Field(min_length=1, max_length=5)]


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4)
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate", ge=date(1980, 1, 1))
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD",
        serialization_alias="baseMSRPUSD"
    )
    top_features: BoundedList[BoundedString] | None = None
    vin: BoundedString
    number_of_doors: int = Field(
        default=4,
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
    )
    registration_country: BoundedString | None = None
    license_plate: BoundedString | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

There are two main changes we are going to make on our model.

First, we want to add an additional field to capture when an automobile was registered. To do so add a field named registration_date that is implemented as follows:

place it right after registration_country in the model
if should be a date object
it should be optional and default to None
it should deserialize from and serialize to the camel case version of the field name
if cannot be earlier than the manufactured_date
just like manufactured_date it should serialize the data to a YYYY/MM/DD format for JSON serialization. (Hint: you do not need to define a second serializer for that field! The syntax is the same as what I showed you when aplying the same decorator validator to multiple fields)
Secondly, we want to ensure that the registration_country only allows values from a pre-determined list of countries.

We are not going to use an enum for this, as there would simply be too many values. Instead we are going to validate the country name against a "database".

For this exercise we are not going to use an actual database, instead you can use the dictionary provided below. The dictionary keys are going to become the accepted "input" value for country names, and each key's value contains a tuple consisting of the country name (properly formatted), and the 3 character country code (we won't use the country code right now, but we will later).

Create a custom validator for registration_country that validates the data being deserialized is one of the keys in that dictionary, and replace the deserialized value with the country name from the first name in the tuple.

For example, if the input data contains:

{
    ...,
    "registrationCountry": "UK",
    ...
}
then, since our "database" gives us this info:

"uk": ("United Kingdom", "GBR")
the deserialized value in our model should become United Kingdom.

Your validator should validate a country name based on the lower-cased and stripped version of the string - i.e. input data such as "UK", "Uk", "uk " should all end up being matched with the key "uk" in the database.

Use an annotated type to do this - name your new annotated type Country.

In [28]:
countries = {
    "australia": ("Australia", "AUS"),
    "canada": ("Canada", "CAN"),
    "china": ("China", "CHN"),
    "france": ("France", "FRA"),
    "germany": ("Germany", "DEU"),
    "india": ("India", "IND"),
    "mexico": ("Mexico", "MEX"),
    "norway": ("Norway", "NOR"),
    "pakistan": ("Pakistan", "PAK"),
    "san marino": ("San Marino", "SMR"),
    "sanmarino": ("San Marino", "SMR"),
    "spain": ("Spain", "ESP"),
    "sweden": ("Sweden", "SWE"),
    "united kingdom": ("United Kingdom", "GBR"),
    "uk": ("United Kingdom", "GBR"),
    "great britain": ("United Kingdom", "GBR"),
    "britain": ("United Kingdom", "GBR"),
    "us": ("United States of America", "USA"),
    "united states": ("United States of America", "USA"),
    "usa": ("United States of America", "USA"),
}

In [29]:
from uuid import UUID

data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4 Competition xDrive",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "topFeatures": ["6 cylinders", "all-wheel drive", "convertible"],
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "us",
    "registrationDate": "2023-06-01",
    "licensePlate": "AAA-BBB"
}

expected_by_alias = {
    'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
    'manufacturer': 'BMW',
    'seriesName': 'M4 Competition xDrive',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'topFeatures': ['6 cylinders', 'all-wheel drive', 'convertible'],
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'United States of America',
    'registrationDate': date(2023, 6, 1),
    'licensePlate': 'AAA-BBB'
}

expected_json_by_alias = '{"id":"c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7","manufacturer":"BMW","seriesName":"M4 Competition xDrive","type":"Convertible","isElectric":false,"manufacturedDate":"2023/01/01","baseMSRPUSD":93300.0,"topFeatures":["6 cylinders","all-wheel drive","convertible"],"vin":"1234567890","numberOfDoors":2,"registrationCountry":"United States of America","registrationDate":"2023/06/01","licensePlate":"AAA-BBB"}'

## Solution
We're going to write a lookup function that will return the country name and code given an input - we'll also perform a case-insensitive search, and strip our string (in case it's not already).

I am going to pre-create a list of valid country names (based on the countries dictionary, so that I can return that information when country validation fails. I am not putting this code into my validator since i do not want to incur the cost of re-generating that list every time the validator is executed.

In [30]:
valid_country_names = sorted(countries.keys())
valid_country_names

['australia',
 'britain',
 'canada',
 'china',
 'france',
 'germany',
 'great britain',
 'india',
 'mexico',
 'norway',
 'pakistan',
 'san marino',
 'sanmarino',
 'spain',
 'sweden',
 'uk',
 'united kingdom',
 'united states',
 'us',
 'usa']

In [31]:
def lookup_country(name: str) -> tuple[str, str]:
    name = name.strip().casefold()

    try:
        return countries[name]
    except KeyError:
        raise ValueError(
            "Unknown country name. "
            f"Country name must be one of: {','.join(valid_country_names)}"
        )

In [32]:
from pydantic import AfterValidator

Country = Annotated[str, AfterValidator(lambda name: lookup_country(name)[0])]

And now, let's use it in our model:

In [33]:
class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4)
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate", ge=date(1980, 1, 1))
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD",
        serialization_alias="baseMSRPUSD"
    )
    top_features: BoundedList[BoundedString] | None = None
    vin: BoundedString
    number_of_doors: int = Field(
        default=4,
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
    )
    registration_country: Country | None = None
    registration_date: date | None = None
    license_plate: BoundedString | None = None

    @field_serializer("manufactured_date", "registration_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

We still have to add a custom validator for registration_date that will check that this date is not earlier than the manufactured_date.

In [34]:
from pydantic import field_validator, ValidationInfo


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4)
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate", ge=date(1980, 1, 1))
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD",
        serialization_alias="baseMSRPUSD"
    )
    top_features: BoundedList[BoundedString] | None = None
    vin: BoundedString
    number_of_doors: int = Field(
        default=4,
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
    )
    registration_country: Country | None = None
    registration_date: date | None = None
    license_plate: BoundedString | None = None

    @field_serializer("manufactured_date", "registration_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

    @field_validator("registration_date")
    @classmethod
    def validate_registration_date(cls, value:date, values: ValidationInfo):
        data = values.data
        if "manufactured_date" in data and data["manufactured_date"] > value:
            raise ValueError("Automobile cannot be registered prior to manufacture date.")
        return value


In [35]:
car = Automobile.model_validate(data)
car

Automobile(id_=UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'), manufacturer='BMW', series_name='M4 Competition xDrive', type_=<AutomobileType.convertible: 'Convertible'>, is_electric=False, manufactured_date=datetime.date(2023, 1, 1), base_msrp_usd=93300.0, top_features=['6 cylinders', 'all-wheel drive', 'convertible'], vin='1234567890', number_of_doors=2, registration_country='United States of America', registration_date=datetime.date(2023, 6, 1), license_plate='AAA-BBB')

In [36]:
assert car.model_dump(by_alias=True) == expected_by_alias

assert car.model_dump_json(by_alias=True) == expected_json_by_alias

In [37]:
bad_data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4 Competition xDrive",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "topFeatures": ["6 cylinders", "all-wheel drive", "convertible"],
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "Lunar Colony",
    "registrationDate": "2022-06-01",
    "licensePlate": "AAA-BBB"
}

In [38]:
from pydantic import ValidationError

try:
    Automobile.model_validate(bad_data)
except ValidationError as ex:
    print(ex)

2 validation errors for Automobile
registrationCountry
  Value error, Unknown country name. Country name must be one of: australia,britain,canada,china,france,germany,great britain,india,mexico,norway,pakistan,san marino,sanmarino,spain,sweden,uk,united kingdom,united states,us,usa [type=value_error, input_value='Lunar Colony', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error
registrationDate
  Value error, Automobile cannot be registered prior to manufacture date. [type=value_error, input_value='2022-06-01', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error


This method also supports an indent parameter - although we normally don't use it when returnin JSON from an API (we try to keep data as compact as possible), here we would want to use it so we can print a legible JSON object.

In [39]:
try:
    Automobile.model_validate(bad_data)
except ValidationError as ex:
    exceptions = ex.json(indent=2)

In [40]:
print(exceptions)

[
  {
    "type": "value_error",
    "loc": [
      "registrationCountry"
    ],
    "msg": "Value error, Unknown country name. Country name must be one of: australia,britain,canada,china,france,germany,great britain,india,mexico,norway,pakistan,san marino,sanmarino,spain,sweden,uk,united kingdom,united states,us,usa",
    "input": "Lunar Colony",
    "ctx": {
      "error": "Unknown country name. Country name must be one of: australia,britain,canada,china,france,germany,great britain,india,mexico,norway,pakistan,san marino,sanmarino,spain,sweden,uk,united kingdom,united states,us,usa"
    },
    "url": "https://errors.pydantic.dev/2.6/v/value_error"
  },
  {
    "type": "value_error",
    "loc": [
      "registrationDate"
    ],
    "msg": "Value error, Automobile cannot be registered prior to manufacture date.",
    "input": "2022-06-01",
    "ctx": {
      "error": "Automobile cannot be registered prior to manufacture date."
    },
    "url": "https://errors.pydantic.dev/2.6/v/value